In [1]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig  

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import yfinance as yf
import pandas as pd 
import numpy as np
import datetime


def data_download(ticker):
    end_date = datetime.datetime.now().date()
    start_date = end_date - datetime.timedelta(days=365)
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

In [5]:
data=data_download('^NSEI')

[*********************100%%**********************]  1 of 1 completed


In [6]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-04-21,6789.250000,6825.450195,6786.899902,6817.649902,6817.649902,111800
2014-04-22,6822.899902,6838.000000,6806.250000,6815.350098,6815.350098,114700
2014-04-23,6823.250000,6861.600098,6820.750000,6840.799805,6840.799805,183000
2014-04-25,6855.799805,6869.850098,6772.850098,6782.750000,6782.750000,154300
2014-04-28,6778.549805,6786.250000,6750.299805,6761.250000,6761.250000,118200


In [7]:
data['10-day SMA'] = data['Adj Close'].rolling(window=5).mean()

In [8]:
data.head(10)

,Open,High,Low,Close,Adj Close,Volume,10-day SMA
Date,,,,,,,
2014-04-21,6789.250000,6825.450195,6786.899902,6817.649902,6817.649902,111800,NaN
2014-04-22,6822.899902,6838.000000,6806.250000,6815.350098,6815.350098,114700,NaN
2014-04-23,6823.250000,6861.600098,6820.750000,6840.799805,6840.799805,183000,NaN
2014-04-25,6855.799805,6869.850098,6772.850098,6782.750000,6782.750000,154300,NaN
2014-04-28,6778.549805,6786.250000,6750.299805,6761.250000,6761.250000,118200,6803.559961
2014-04-29,6769.000000,6779.700195,6708.649902,6715.250000,6715.250000,115500,6783.079980
2014-04-30,6724.950195,6780.149902,6656.799805,6696.399902,6696.399902,161100,6759.289941
2014-05-02,6709.950195,6737.649902,6689.500000,6694.799805,6694.799805,114300,6730.089941
2014-05-05,6681.649902,6741.049805,6680.450195,6699.350098,6699.350098,105000,6713.409961


In [2]:
tokenizer_sentiment = AutoTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
model_sentiment = AutoModelForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis", num_labels=3)
lora_config = LoraConfig.from_pretrained('models/FinancialBert')
model_sentiment = get_peft_model(model_sentiment, lora_config)

In [3]:
text='Raute reported a loss per share of EUR 0.86 for the first half of 2009 , against EPS of EUR 0.74 in the corresponding period of 2008.'
encoded_input = tokenizer_sentiment(text,padding=True,truncation=True ,return_tensors='pt')
output = model_sentiment(**encoded_input)
scores = output[0][0].detach().numpy()
# scores = softmax(scores)

In [4]:
scores

array([ 3.9228015, -2.6699772, -0.5829254], dtype=float32)